In [1]:
from pyorthanc import Orthanc, Instance, Patient, Study
import tempfile
import pickle
import base64
import glob
import os.path
import urllib
import zlib
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from typing import List, Dict

import requests
from tqdm import tqdm
from urllib3.exceptions import ProtocolError

import SimpleITK as sitk
import glob

import  os
import subprocess
import pandas as pd
import numpy as np
import unicodedata, re
from datetime import datetime
from collections import defaultdict

import pyorthanc

pd.options.mode.chained_assignment = None  # default='warn'



In [2]:
!pip install openpyxl


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# Initialize orthanc client
orthanc_client = Orthanc('http://localhost:8042')
orthanc_client.setup_credentials('dev-user-alta', 'SyTP&8JbKFx@a6R65^sE`Z$') 

# test conntection
patient_ids = orthanc_client.get_patients()
len(patient_ids)

10003

In [4]:
# Load current meta from PACs and match mit ALTA IDs
dicom_meta_data = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/metadata/meta_pacs_13056_20240126.csv", sep=";")
# prepare dicom_meta data
dicom_meta_data['StudyDate'] = pd.to_datetime(dicom_meta_data['StudyDate'], format="%Y%m%d")
dicom_meta_data_alta = pd.read_csv("/data/oleksii/datasets/ALTA-Lesions-Dataset/classification/clinical_param/meta_pacs_alta_id.csv", sep=";")
dicom_meta_data_alta = dicom_meta_data_alta[['StudyInstanceUID', 'ALTA ID']]
dicom_meta_data_alta_merged = pd.merge(
    dicom_meta_data,
    dicom_meta_data_alta,
    how="left",
    on='StudyInstanceUID',
)
cond = dicom_meta_data_alta_merged["ALTA ID"].isna() & dicom_meta_data_alta_merged["InstitutionName"].str.lower().str.contains('|'.join(["lumiani", "alta"]))
dicom_meta_data_alta_merged.loc[cond, "ALTA ID"] = dicom_meta_data_alta_merged[cond]['PatientID']
dicom_meta_data_alta_merged['ALTA ID str'] = dicom_meta_data_alta_merged['ALTA ID'].str.rjust(12, "0")
dicom_meta_data_alta_merged['PatientID str'] = dicom_meta_data_alta_merged['PatientID'].str.rjust(12, "0")
dicom_meta_data_alta_merged

,InstitutionName,PatientAge,PatientBirthDate,PatientID,PatientName,PatientSize,PatientWeight,StudyDate,StudyInstanceUID,study_orthanc_id,ALTA ID,ALTA ID str,PatientID str
0,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",055Y,19660416,26586,Zirngibl^Thomas,1.87,93.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,786714b5-a6309313-214532e5-e92d1dff-308695f6,26586,000000026586,000000026586
1,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",068Y,19530401,31906,Ständer^Ralf,1.80,135.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,e27d17d9-47c79ade-2218d504-e26738cd-568156b3,31906,000000031906,000000031906
2,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",068Y,19530401,31906,Ständer^Ralf,1.80,135.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,21915507-0546865f-dfde6bc5-77369bc3-bb431e29,31906,000000031906,000000031906
3,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",067Y,19541206,32266,Schep^Jan,1.93,95.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,69abb26e-29ce4d91-7078e646-fe2802b9-993e25d2,32266,000000032266,000000032266
4,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",080Y,19410529,32381,Winkler^Thomas^^Dr.med.,1.80,120.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,8db58a55-a429d9f1-d9332029-aa67c922-86db6de5,32381,000000032381,000000032381
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13127,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",066Y,19550221,17716,Opheys^Michael^^Dr. med.,1.73,76.0,2021-04-14,1.3.12.2.1107.5.2.19.145132.300000210414061351...,198bb039-cc51cde6-e3be1375-9f23453e-120c31f1,17716,000000017716,000000017716
13128,medneo Diagnostikzentrum Charlottenburg,065Y,19560326,aaa136966524,Herklotz^Michael Werner,1.89,90.0,2021-04-17,1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.8208...,e3015f04-6c51c4bf-01e1d90a-d31afc03-8472ec42,NaN,NaN,aaa136966524
13129,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",070Y,19501110,25030,Roache^Donald,1.88,88.0,2021-04-26,1.3.12.2.1107.5.2.19.145132.300000210426063458...,d47141ad-8aa12d54-7de734f8-b7d670a4-356c1997,25030,000000025030,000000025030
13130,Radiologie im Zentrum - MRT - Privatpraxis,057Y,19630616,02960-21,Deifuß^Reinhard,1.72,78.0,2021-03-24,1.3.12.2.1107.5.2.20.72016.3000002103240740566...,9daa3cd4-3885cc54-f1462991-a5246032-eec5599a,NaN,NaN,000002960-21


### TODO:
[ ] Retrierve series with orthanc id

[ ] Pack classes ids into the jsons    



## Get data for Lucas

In [6]:
glob.glob(os.path.join('/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/preprocessed', 'fold_ids_seed*.pickle'))

['/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/preprocessed/fold_ids_seed_42.pickle',
 '/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/preprocessed/fold_ids_seed_42_str.pickle',
 '/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/preprocessed/fold_ids_seed_6000_str.pickle']

In [7]:
with open('/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/preprocessed/fold_ids_seed_6000_str.pickle', 'rb') as handle:
    pkl = pickle.load(handle)
    
test_set = pd.DataFrame.from_records(pkl['test_data']['data_test'])
train_set = pd.DataFrame.from_records(pkl['train_data']['data_train'])
train_set['split'] = 'train'
test_set['split'] = 'test'
lesion_dataset_1074 = pd.concat([train_set, test_set])
lesion_dataset_1074 = lesion_dataset_1074.join(lesion_dataset_1074['label'].str.split('_', n=2, expand=True).rename(columns={0:'PatientID', 1:'StudyDate'}))
lesion_dataset_1074.to_csv('lesion_dataset_1074.csv', index=False)
lesion_dataset_1074 = pd.concat([train_set, test_set])
lesion_dataset_1074

,label,t2w,adc,dwi,split
0,000000000786_20201026_tra_seg.nii.gz,000000000786_20201026_tra_t2w.nii.gz,000000000786_20201026_tra_adc_reg.nii.gz,000000000786_20201026_tra_50_dwi_reg.nii.gz,train
1,000000000892_20120914_tra_seg.nii.gz,000000000892_20120914_tra_t2w.nii.gz,000000000892_20120914_tra_adc_reg.nii.gz,000000000892_20120914_tra_50_dwi_reg.nii.gz,train
2,000000001838_20190916_tra_seg.nii.gz,000000001838_20190916_tra_t2w.nii.gz,000000001838_20190916_tra_adc_reg.nii.gz,000000001838_20190916_tra_50_dwi_reg.nii.gz,train
3,000000002593_20210622_tra_seg.nii.gz,000000002593_20210622_tra_t2w.nii.gz,000000002593_20210622_tra_adc_reg.nii.gz,000000002593_20210622_tra_50_dwi_reg.nii.gz,train
4,000000002820_20120223_tra_seg.nii.gz,000000002820_20120223_tra_t2w.nii.gz,000000002820_20120223_tra_adc_reg.nii.gz,000000002820_20120223_tra_50_dwi_reg.nii.gz,train
...,...,...,...,...,...
175,000000030320_20210505_tra_seg.nii.gz,000000030320_20210505_tra_t2w.nii.gz,000000030320_20210505_tra_adc_reg.nii.gz,000000030320_20210505_tra_50_dwi_reg.nii.gz,test
176,000000030585_20210612_tra_seg.nii.gz,000000030585_20210612_tra_t2w.nii.gz,000000030585_20210612_tra_adc_reg.nii.gz,000000030585_20210612_tra_50_dwi_reg.nii.gz,test
177,000000030681_20210625_tra_seg.nii.gz,000000030681_20210625_tra_t2w.nii.gz,000000030681_20210625_tra_adc_reg.nii.gz,000000030681_20210625_tra_50_dwi_reg.nii.gz,test
178,000000030759_20210705_tra_seg.nii.gz,000000030759_20210705_tra_t2w.nii.gz,000000030759_20210705_tra_adc_reg.nii.gz,000000030759_20210705_tra_50_dwi_reg.nii.gz,test


In [651]:
def get_alta_lesion_ids():
    
#     dicom_meta_data_alta_merged = get_meta_with_correct_alta_ids()
    
    # get old segmentations 
    path = "/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/segmentations"
    seg_cases = glob.glob(os.path.join(path, "*/*/*/Segmentation-label-tra-final.seg.nrrd"))
    # retrieve corresponding orthanc studies
    studies = []
    for idx, case in enumerate(seg_cases[:]):
        patient_id, study_date = case.split("/")[-4:-2]
        # find patient in orthanc database     
        patient = dicom_meta_data_alta_merged[dicom_meta_data_alta_merged['ALTA ID str'] == patient_id]
        studies_ids = patient[['StudyInstanceUID', 'StudyDate']].to_dict(orient='records')
        if studies_ids:
            # iterate over the studies of the patients
            c = 0             
            for study_id in studies_ids: 
                s = orthanc_client.post_tools_lookup(data=study_id['StudyInstanceUID'])
                
                if study_id['StudyDate'].to_pydatetime().timetuple()[:3] == datetime.strptime(study_date, '%Y%m%d').timetuple()[:3]:
                    # print('there is a match ')
                    studies.append({"study_orthanc_id": s[0]['ID'], "seg": case})
                    c = c + 1
            if c == 0:
                studies.append({"study_orthanc_id": None, "seg": case})                
                
                    
        else: 
            studies.append({"study_orthanc_id": None, "seg": case})
    
    return studies

In [5]:
def get_oid_from_uid(uid):
    study_candidates = orthanc_client.post_tools_lookup(data=uid)
    for i in study_candidates:
        if i['Type'] == 'Study':
            return i['ID']
    return None


In [6]:
study_time_orthanc = datetime.strptime(study.get_main_information()['MainDicomTags']['StudyDate'], '%Y%m%d')
study_time_record = row['Study Date']
diff = study_time_orthanc
- study_time_record

NameError: name 'study' is not defined

In [973]:
# cases = pd.read_csv("cases.csv", sep=',')
cases = pd.read_csv("meta_data_lesion/missing_images_to_existing_lesions_lucas_final.csv", sep=';')
for i, row in cases.iterrows():
    if row['StudyInstanceUID'] != '#NV':
        oid = get_oid_from_uid(row['StudyInstanceUID'])
        print(oid, row['StudyInstanceUID'])
        cases.loc[i, "study_orthanc_id"] = oid

cases = cases[~cases['study_orthanc_id'].isna()]
cases.to_csv("meta_data_lesion/missing_images_to_existing_lesions_lucas_final_oid.csv", sep=";")

752f842e-c7f224d4-1611d32a-33225378-b80226f0 1.2.40.0.13.0.11.4050.2.2020022191.389551.20200605120009
74928762-cdd6561b-6c0ca571-b28bef3e-449d46bc 1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.22602722.16746863
b19382d9-8130f9ed-a7a76f97-9bffab96-2ca62179 1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.72967935.20026123
e52d688b-98dac1de-c5996746-d2fbdec7-3dc8ed87 1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.97931534.19749663
59365e28-7c59cd88-a88065ba-65e55b52-ef0a7cd7 1.2.276.0.38.1.1.1.22177.20200525171057.66445132
f4410913-d5d6cd8f-e4583fcb-67d40ffb-d369e9f4 1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.34346146.29830013
03c6c72a-b15a91c9-da2fb07a-084bf812-0e674895 1.3.12.2.1107.5.2.19.145132.30000021033006054022700000028
9f03ccf8-3d78f90f-992be5e2-47baf4ad-477076ba 1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.64591053.19858393
c5adbcce-8d686679-a354b0a9-b9cc2212-857a534a 1.2.276.0.37.1.69.199810.80282088
6c946c2f-56c29fad-dc77b49b-2580e0bd-6e5b050a 1.2.840.113619.6.95.31.0.3.4.1.6013.13.10830660
bcdd

In [1000]:
seq_map = pd.read_csv('meta_data_lesion/sequence_mapping_13056_studies_20240124.csv', sep=';')
cases_merged = pd.merge(cases_avail, seq_map, on='study_orthanc_id', how='left')
# get_cases where something is missing again 
cases_merged_all = cases_merged[~(cases_merged['t2w_tra_id'].isna() | cases_merged['adc_tra_id'].isna() | cases_merged['dwi_tra_id'].isna())]


In [1003]:
cases_merged_all

,seg_path,ALTA_ID,StudyDate_x,Datum,DICOM ID,Matching ID,StudyInstanceUID,Stage,oid,study_orthanc_id,...,adc_cor_sd,dwi_tra_id,dwi_tra_sd,dwi_sag_id,dwi_sag_sd,dwi_cor_id,dwi_cor_sd,Manufacturer,PatientID,StudyDate_y
0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,28548,20200610,10.06.20,90456479,"9,04565E+12",1.2.40.0.13.0.11.4050.2.2020022191.389551.2020...,AWS,NaN,752f842e-c7f224d4-1611d32a-33225378-b80226f0,...,NaN,7f4a2120-da9f87f7-36047360-84fdaf77-64407591,ep2d_diff_b50_400_1000_tra(prostata)_tracew_df...,NaN,NaN,NaN,NaN,siemens,90456479,20200610.0
1,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,25979,20190720,20.07.19,aaa209137120,aaa20913712043666,1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.2260...,PACS,74928762-cdd6561b-6c0ca571-b28bef3e-449d46bc,74928762-cdd6561b-6c0ca571-b28bef3e-449d46bc,...,NaN,d7d021f9-23f1aa1f-4b95739b-46be66fa-90c04e33,ep2d_diff_b50_500_1000_tra_tracew_dfc_mix,NaN,NaN,NaN,NaN,siemens,aaa209137120,20190720.0
2,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,26622,20191023,23.10.19,aaa216170542,aaa21617054243761,1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.7296...,PACS,b19382d9-8130f9ed-a7a76f97-9bffab96-2ca62179,b19382d9-8130f9ed-a7a76f97-9bffab96-2ca62179,...,ep2d_diff_b50_500_1000_cor_adc_dfc,626c83c6-f8c76571-2fadc3cb-9d945338-da339430,ep2d_diff_b50_500_1000_tra_tracew_dfc_mix,NaN,NaN,76745a59-8a41a621-4be80d20-6de90846-a7a2d779,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,aaa216170542,20191023.0
3,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,26557,20191016,16.10.19,aaa202772277,aaa20277227743754,1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.9793...,PACS,e52d688b-98dac1de-c5996746-d2fbdec7-3dc8ed87,e52d688b-98dac1de-c5996746-d2fbdec7-3dc8ed87,...,ep2d_diff_b50_500_1000_cor_adc_dfc,cdb99046-59db2dd9-a13c81fe-73db78f1-20eb7712,ep2d_diff_b50_500_1000_tra_tracew_dfc_mix,NaN,NaN,23b8c6f2-bfcf042f-86b3cd06-0df60ebc-94e7e53b,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,aaa202772277,20191016.0
4,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,28525,20200708,08.07.20,40267811,"4,02678E+12",1.2.276.0.38.1.1.1.22177.20200525171057.66445132,AWS,NaN,59365e28-7c59cd88-a88065ba-65e55b52-ef0a7cd7,...,NaN,ad8fe6cf-418a29a3-63b91484-3588dc85-55fa7fbb,t2-tse-epi-tra_tracew,NaN,NaN,NaN,NaN,siemens,0040267811,20200708.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,26090,20190806,06.08.19,26090,2609043683,1.3.12.2.1107.5.2.19.145132.300000190806055534...,PACS,17a34615-a0695eb3-2c570128-5e8bc933-b3370aad,17a34615-a0695eb3-2c570128-5e8bc933-b3370aad,...,ep2d_diff_b50_500_1000_cor_adc_dfc,4f73643a-70d52ccf-915cda8d-1b87f2cf-bd074cdc,ep2d_diff_b50_500_1000_tra_tracew_dfc,NaN,NaN,54b5f041-e9c3f430-e8f3bb43-750d4c91-7f3a3210,ep2d_diff_b50_500_1000_cor_tracew_dfc,siemens,26090,20190806.0
95,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,29841,20210310,10.03.21,aaa254832389,aaa25483238944265,1.3.6.1.4.1.28841.1.10.20.1.9117.17435148.3330...,PACS,02213065-3d88735e-9c90df2b-60e5edfc-d0b10c4c,02213065-3d88735e-9c90df2b-60e5edfc-d0b10c4c,...,NaN,70694bb9-9bff04cb-3b362a21-881cbafa-9fb7fca7,ep2d_diff_b50_500_1000_tra_tracew_dfc_mix,NaN,NaN,NaN,NaN,siemens,aaa254832389,20210310.0
96,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,25730,20190606,06.06.19,11280115,"1,12801E+12",1005.99.332.2647938.6,PACS,cdb4a16a-e92b4e2c-e5414d01-a39007f6-f3f9b64f,cdb4a16a-e92b4e2c-e5414d01-a39007f6-f3f9b64f,...,NaN,eb517453-7006568a-9b39ccec-e47b5444-1d52f7e3,ep2d_diff_tra_b50-400-800_tracew_dfc,NaN,NaN,NaN,NaN,siemens,11280115,20190606.0
97,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,28501,20200805,05.08.20,77649,7764944048,1.2.276.0.37.1.262.201304.236196,PACS,adf6c582-a4d4db28-27176d0e-d27c0b32-b9befbb0,adf6c582-a4d4db28-27176d0e-d27c0b32-b9befbb0,...,NaN,ef2fe10e-eec75902-d935e2c7-a462683d-0a5cce42,ep2d_diff_b50_500_1000_tra_p2_tracew_dfc_mix,NaN,NaN,NaN,NaN,siemens,77649,20200805.0


In [757]:
dicom_meta_data_alta_merged

,InstitutionName,PatientAge,PatientBirthDate,PatientID,PatientName,PatientSize,PatientWeight,StudyDate,StudyInstanceUID,ALTA ID,ALTA ID str,PatientID str
0,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",055Y,19660416,26586,Zirngibl^Thomas,1.87,93.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,26586,000000026586,000000026586
1,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",068Y,19530401,31906,Ständer^Ralf,1.80,135.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,31906,000000031906,000000031906
2,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",068Y,19530401,31906,Ständer^Ralf,1.80,135.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,31906,000000031906,000000031906
3,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",067Y,19541206,32266,Schep^Jan,1.93,95.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,32266,000000032266,000000032266
4,"ALTA Klinik Bielefeld, Dr. med. A. Lumiani",080Y,19410529,32381,Winkler^Thomas^^Dr.med.,1.80,120.0,2022-02-01,1.3.12.2.1107.5.2.19.145132.300000220201065038...,32381,000000032381,000000032381
...,...,...,...,...,...,...,...,...,...,...,...,...
13095,medneo Diagnostikzentrum Hamburg,072Y,19500805,mde10008857,Lauber^Thomas,1.70,73.0,2023-06-07,1.2.826.0.1.5029338.202306071043204929.10425,NaN,NaN,0mde10008857
13096,NaN,062Y,19600726,36079800,Pfeifer^Siegfried,1.89,83.0,2023-06-01,1.2.276.0.38.1.1.1.3150.20230526081802.48783562,NaN,NaN,000036079800
13097,ALTA Klinik,057Y,19650723,41904,Rüter^Dirk,1.80,79.0,2023-06-19,1.3.12.2.1107.5.2.19.145132.300000230619072446...,41904,000000041904,000000041904
13098,ALTA Klinik,063Y,19600203,15220,Meiners^Torsten^^Dr. med.,1.82,84.0,2023-06-19,1.3.12.2.1107.5.2.19.145132.300000230619072446...,15220,000000015220,000000015220


In [767]:
dicom_meta_data_alta_merged[dicom_meta_data_alta_merged['PatientID'] == '17844']

,InstitutionName,PatientAge,PatientBirthDate,PatientID,PatientName,PatientSize,PatientWeight,StudyDate,StudyInstanceUID,ALTA ID,ALTA ID str,PatientID str
3093,ALTA Klinik Bielefeld,070Y,19470328,17844,Riechey^Gunter^^Dr.,1.63,70.0,2017-08-14,1.3.12.2.1107.5.2.19.145132.300000170814060311...,17844,000000017844,000000017844
7005,Dr. med. A. Lumiani,067Y,19470328,17844,Riechey^Gunter^^Dr.,1.63,70.0,2015-01-27,1.3.12.2.1107.5.2.30.59050.3000001501270713463...,17844,000000017844,000000017844
7354,Dr. med. A. Lumiani,069Y,19470328,17844,Riechey^Gunter^^Dr.,1.63,70.0,2016-12-15,1.3.12.2.1107.5.2.30.59050.3000001612150710156...,17844,000000017844,000000017844


In [743]:
# Matching existing lesion with the orthanc database

path = "/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/segmentations"
seg_cases = glob.glob(os.path.join(path, "*/*/*/Segmentation-label-tra-final.seg.nrrd"))
# retrieve corresponding orthanc studies
studies = []
d = {}
for idx, case in enumerate(seg_cases):
    d[case] = None
    
    patient_id, study_date = case.split("/")[-4:-2]
    # find patient in orthanc database     
    patient = dicom_meta_data_alta_merged[dicom_meta_data_alta_merged['ALTA ID str'] == patient_id]
    studies_ids = patient[['StudyInstanceUID', 'StudyDate']].to_dict(orient='records')
    if studies_ids:
        # iterate over the studies of the patients
        c = 0             
        for study_row in studies_ids: 
            study_oid = get_oid_from_uid(study_row['StudyInstanceUID'])            
            if study_oid is not None: 
                study = Study(study_oid, orthanc_client)
                study_time_orthanc = datetime.strptime(study.get_main_information()['MainDicomTags']['StudyDate'], '%Y%m%d')
                study_time_seg = datetime.strptime(study_date, '%Y%m%d')
                diff = study_time_orthanc - study_time_seg
                if diff.days == 0:
                    d[case] = study_oid
            else:
                print("None")
# Excluding the missing images dataset and sending it to Lucas
alta_lesion_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
alta_lesion_df.columns = ['seg_path', 'study_orthanc_id']


In [745]:
len(seg_cases)

1183

In [794]:
alta_lesion_df_final = alta_lesion_df[~alta_lesion_df['study_orthanc_id'].isna()]

In [ ]:
# TODO: MOVE SEGMENATIONS TO THIS BATCH and match class indices

In [805]:
# Excluding the missing images dataset and sending it to Lucas
# alta_lesion_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
# alta_lesion_df.columns = ['seg_path', 'study_orthanc_id']
# alta_lesion_df_missing_img = alta_lesion_df[alta_lesion_df['study_orthanc_id'].isna()]
# alta_lesion_df_missing_img = alta_lesion_df_missing_img.join(alta_lesion_df_missing_img['seg_path'].str.split('/', n=-1, expand=True).rename(columns={7:'ALTA_ID', 8:'StudyDate'}))
# alta_lesion_df_missing_img.drop(columns=['study_orthanc_id', 0, 1, 2, 3, 4, 5, 6, 9, 10], inplace=True)
# alta_lesion_df_missing_img.to_csv('missing_images_to_existing_lesions.csv', sep=';', index=False)
# alta_lesion_df_missing_img

# or make it cleaner while relying on the Actual extracted data
fetched_data = os.listdir("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-fist-batch-2/")
alta_lesion_df_missing_img = alta_lesion_df[~alta_lesion_df['study_orthanc_id'].isin(fetched_data)]
alta_lesion_df_missing_img = alta_lesion_df_missing_img.join(alta_lesion_df_missing_img['seg_path'].str.split('/', n=-1, expand=True).rename(columns={7:'ALTA_ID', 8:'StudyDate'}))
alta_lesion_df_missing_img.drop(columns=['study_orthanc_id', 0, 1, 2, 3, 4, 5, 6, 9, 10], inplace=True)
alta_lesion_df_missing_img.to_csv('missing_images_to_existing_lesions.csv', sep=';', index=False)
alta_lesion_df_missing_img

,seg_path,ALTA_ID,StudyDate
19,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000029833,20201230
31,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000025979,20190720
41,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000026622,20191023
46,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000026557,20191016
48,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000017844,20201022
...,...,...,...
1150,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000030055,20210329
1162,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000029757,20210322
1165,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000014541,20210311
1168,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000028501,20200805


## Prepare table for old lesions

In [1111]:
def get_meta_with_correct_alta_ids():
    # Load current meta from PACs and match mit ALTA IDs
    dicom_meta_data = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/meta_pacs_20230719.csv", sep=";")
    # prepare dicom_meta data
    dicom_meta_data['StudyDate'] = pd.to_datetime(dicom_meta_data['StudyDate'], format="%Y%m%d")
    dicom_meta_data_alta = pd.read_csv("/data/oleksii/datasets/ALTA-Lesions-Dataset/classification/clinical_param/meta_pacs_alta_id.csv", sep=";")
    dicom_meta_data_alta = dicom_meta_data_alta[['StudyInstanceUID', 'ALTA ID']]
    dicom_meta_data_alta_merged = pd.merge(
        dicom_meta_data,
        dicom_meta_data_alta,
        how="left",
        on='StudyInstanceUID',
    )
    cond = dicom_meta_data_alta_merged["ALTA ID"].isna() & dicom_meta_data_alta_merged["InstitutionName"].str.lower().str.contains('|'.join(["lumiani", "alta"]))
    dicom_meta_data_alta_merged.loc[cond, "ALTA ID"] = dicom_meta_data_alta_merged[cond]['PatientID']
    dicom_meta_data_alta_merged['ALTA ID str'] = dicom_meta_data_alta_merged['ALTA ID'].str.rjust(12, "0")
    dicom_meta_data_alta_merged['PatientID str'] = dicom_meta_data_alta_merged['PatientID'].str.rjust(12, "0")
    return dicom_meta_data_alta_merged            

def get_alta_lesion_ids():
    dicom_meta_data_alta_merged = get_meta_with_correct_alta_ids()
    
    path = "/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/segmentations"
    seg_cases = glob.glob(os.path.join(path, "*/*/*/Segmentation-label-tra-final.seg.nrrd"))
    # retrieve corresponding orthanc studies
    d = {}
    for _, case in enumerate(seg_cases):
        # populate None by default 
        d[case] = None
        
        patient_id, study_date = case.split("/")[-4:-2]
        # find patient in orthanc database     
        patient = dicom_meta_data_alta_merged[dicom_meta_data_alta_merged['ALTA ID str'] == patient_id]
        studies_ids = patient[['StudyInstanceUID', 'StudyDate']].to_dict(orient='records')
        if studies_ids:
            # iterate over the studies of the patients
            for study_row in studies_ids: 
                study_oid = get_oid_from_uid(study_row['StudyInstanceUID'])            
                if study_oid is not None: 
                    study = Study(study_oid, orthanc_client)
                    study_time_orthanc = datetime.strptime(study.get_main_information()['MainDicomTags']['StudyDate'], '%Y%m%d')
                    study_time_seg = datetime.strptime(study_date, '%Y%m%d')
                    diff = study_time_orthanc - study_time_seg
                    if diff.days == 0:
                        d[case] = study_oid

    # Excluding the missing images dataset and sending it to Lucas
    alta_lesion_df = pd.DataFrame.from_dict(d, orient='index').reset_index()
    alta_lesion_df.columns = ['seg_path', 'study_orthanc_id']   
    
    return alta_lesion_df

In [1161]:
# converting old dataset into the new structure 
alta_lesion_batch = get_alta_lesion_ids()
alta_lesion_batch = alta_lesion_batch[~alta_lesion_batch['study_orthanc_id'].isna()]
alta_lesion_batch['relative_path'] = alta_lesion_batch.apply(lambda x: '/'.join([x['seg_path'].split('/')[-4], x['seg_path'].split('/')[-3]]), axis=1)

# exclude cased from 
cases_avail = pd.read_csv('meta_data_lesion/missing_images_to_existing_lesions_lucas_final_oid.csv', sep=';')
cases_avail = cases_avail[['seg_path','study_orthanc_id']]
cases_avail['relative_path'] = cases_avail.apply(lambda x: '/'.join([x['seg_path'].split('/')[-4], x['seg_path'].split('/')[-3]]), axis=1)

In [8]:
lesion_dataset_classes = pd.read_csv('/data/oleksii/datasets/ALTA-Lesions-Dataset/segmentation/lesion_dataset.csv', sep=";")
# alta_lesion = pd.concat([cases_avail, alta_lesion_batch])
# lesion_dataset_merged = pd.merge(alta_lesion, lesion_dataset_classes, on='relative_path', how='left')
##TODO!! COmplete


In [9]:
lesion_dataset_classes

,ALTA_ID,StudyDate,Gleason,OrderN,Befund als Wort,source,Old_ID,Composition,relative_path,ALTA_ID_CORRECT,relative_path_depr,segm_name,has_dicom,class_seg,class_id,DICOM_ID
0,786,20201026,6,1.0,karzinom,Batch007_Cancer.csv,NaN,3+3,000000000786/20201026,NaN,000000000786/20201026,Segment_1,True,cancer,1,NaN
1,892,20120914,prostatitis,1.0,NaN,alta-lesion-prostatitis,NaN,NaN,000000000892/20120914,NaN,000000000892/20120914,NaN,True,prostatitis,7,NaN
2,1838,20190916,7a,1.0,NaN,alta-lesion-cancer,NaN,(3 + 4),000000001838/20190916,NaN,000000001838/20190916,Segment_1,True,cancer,2,NaN
3,2593,20210622,high grade pin,1.0,high grade pin,Batch004_BiopsyJuly_and_PiRads5_false_positive...,NaN,NaN,000000002593/20210622,NaN,000000002593/20210622,Segment_1,True,cancer,9,NaN
4,2593,20210622,6,2.0,karzinom,Batch004_BiopsyJuly_and_PiRads5_false_positive...,NaN,3+3,000000002593/20210622,NaN,000000002593/20210622,Segment_2,True,cancer,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,29997,24.03.21,prostatitis,2.0,prostatitis,Batch12,NaN,NaN,000000029997/20210324,NaN,NaN,NaN,NaN,prostatitis,7,aaa256045936
1979,30059,30.03.21,prostatitis,1.0,prostatitis,Batch12,NaN,NaN,000000030059/20210330,NaN,NaN,NaN,NaN,prostatitis,7,30059
1980,30059,30.03.21,prostatitis,2.0,prostatitis,Batch12,NaN,NaN,000000030059/20210330,NaN,NaN,NaN,NaN,prostatitis,7,30059
1981,30086,21.04.21,prostatitis,1.0,prostatitis,Batch12,NaN,NaN,000000030086/20210421,NaN,NaN,NaN,NaN,prostatitis,7,aaa258451419


In [1165]:
lesion_dataset_merged = lesion_dataset_merged[['study_orthanc_id', 'Gleason', 'OrderN', 'seg_path', 'relative_path']]
lesion_dataset_merged.rename(columns={'Gleason': 'class', 'OrderN': 'segment_id'}, inplace=True)
# lesion_dataset_merged['region'] = None
# lesion_dataset_merged['zone'] = None
lesion_dataset_merged['segment_id'] = lesion_dataset_merged['segment_id'].astype(int, errors='ignore')

In [1226]:
# studies with issues
with_nan_classes = lesion_dataset_merged[lesion_dataset_merged['segment_id'].isna()]

In [1167]:
lesion_dataset_merged['class'].replace('Chronic inflammation', 'CI', inplace=True)
lesion_dataset_merged['class'].replace('chronische Entzündung', 'CI', inplace=True)
lesion_dataset_merged['class'].replace('prostatitis + high grade pin', 'CI', inplace=True)
lesion_dataset_merged['class'].replace('prostatitis', 'CI', inplace=True)

lesion_dataset_merged['class'].replace('No tumor', 'N/A', inplace=True)
lesion_dataset_merged['class'].replace('tumorfreies prostatagewebe', 'N/A', inplace=True)
lesion_dataset_merged['class'].replace('High-grade PIN', 'HighPIN', inplace=True)
lesion_dataset_merged['class'].replace('high grade pin', 'HighPIN', inplace=True)

lesion_dataset_merged['class'].replace('6', 'GS6', inplace=True)
lesion_dataset_merged['class'].replace('7a', 'GS7a', inplace=True)
lesion_dataset_merged['class'].replace('7b', 'GS7b', inplace=True)
lesion_dataset_merged['class'].replace('8', 'GS8', inplace=True)
lesion_dataset_merged['class'].replace('9', 'GS9', inplace=True)
lesion_dataset_merged['class'].replace('10', 'GS10', inplace=True)


In [1168]:
lesion_dataset_merged

,study_orthanc_id,class,segment_id,seg_path,relative_path
0,752f842e-c7f224d4-1611d32a-33225378-b80226f0,GS7a,1.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000028548/20200610
1,752f842e-c7f224d4-1611d32a-33225378-b80226f0,CI,2.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000028548/20200610
2,74928762-cdd6561b-6c0ca571-b28bef3e-449d46bc,GS9,1.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000025979/20190720
3,74928762-cdd6561b-6c0ca571-b28bef3e-449d46bc,GS9,2.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000025979/20190720
4,74928762-cdd6561b-6c0ca571-b28bef3e-449d46bc,GS9,3.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000025979/20190720
...,...,...,...,...,...
1980,e8ce980a-fef25970-c62c8a53-0274e78a-323d5af6,CI,1.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000013136/20120628
1981,5238a352-2fd2314c-1df0bf94-99b8616c-45685809,CI,1.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000011863/20111130
1982,5238a352-2fd2314c-1df0bf94-99b8616c-45685809,CI,2.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000011863/20111130
1983,5238a352-2fd2314c-1df0bf94-99b8616c-45685809,CI,3.0,/data/oleksii/datasets/ALTA-Lesions-Dataset/se...,000000011863/20111130


In [1171]:
group = lesion_dataset_merged.groupby(['study_orthanc_id'])
new_df = group.agg({'class': lambda x: ','.join([str(i)for i in x]), 
#                     'region': lambda x: ','.join([str(i)for i in x]), 
#                     'zone': lambda x: ','.join([str(i)for i in x]),
                    'segment_id': lambda x: ','.join([str(i)for i in x]),
#                     'seg_path': lambda x: x,
                   })
new_df['count'] = group.count()['segment_id']
new_df = new_df.reset_index()

In [1231]:
real_class_available = new_df[~new_df['study_orthanc_id'].isin(with_nan_classes['study_orthanc_id'])]
real_class_available

,study_orthanc_id,class,segment_id,count
0,00af801a-b4d4a2cc-4c6166f6-912149ab-e4b49ac0,"CI,CI","1.0,2.0",2
1,00df993f-5497a492-9649310b-eb941436-97d98435,GS7b,1.0,1
2,0124a5db-310aa3f6-d531bc99-548a5800-36b78f65,"N/A,N/A","1.0,2.0",2
3,0127cc59-d6839c0c-d0dc37ba-e4f4ae01-ed66fb06,N/A,1.0,1
4,013282a0-2de4fe3b-d4ffe513-d2db5900-a37d8fcf,N/A,1.0,1
...,...,...,...,...
1167,ff62e719-e33423be-49a963b8-2d80a687-58e3d65c,"GS9,HighPIN,HighPIN","1.0,2.0,3.0",3
1168,ff707595-1e66b158-1f36ab68-79289751-c171bb7f,"GS9,CI","1.0,2.0",2
1169,ffcee9ca-5aa578e7-fdeac08a-949739d1-728027eb,CI,1.0,1
1170,ffcf7f45-fd4a6512-b553774b-5f0de689-c17cfe46,GS7b,1.0,1


In [1232]:
no_real_class_available = new_df[new_df['study_orthanc_id'].isin(with_nan_classes['study_orthanc_id'])]
no_real_class_available

,study_orthanc_id,class,segment_id,count
60,0f3b3042-cfef7f84-dda03ae9-4a8ed643-6bcccb9d,nan,nan,0
70,111229c5-38c1bcb2-46d6339e-e6ca4e17-22433de8,nan,nan,0
182,2f64fd47-c6f5b88c-f0238c4f-9d2c1b6e-5f2de2b9,nan,nan,0
187,3044adae-77ad37b0-7858c063-a7f31e44-89d5a03d,nan,nan,0
219,354c7851-b56b4456-3f60882b-4a877adc-08a465e5,nan,nan,0
223,3629c2e7-c694bdb3-297f116f-43f47407-9067e347,nan,nan,0
255,3c7c95f5-de3f1cfe-b738cc81-c7aff65b-99e0444a,nan,nan,0
303,47a67031-8a58a142-a71c244b-234bef8a-b4119554,nan,nan,0
475,6e45479e-8341f99c-0983ddfc-37b1c61d-71fd0889,nan,nan,0
499,73f17b78-e8177e2b-b82f62c1-ead8a3d6-626e9fb9,nan,nan,0


### Get classes from mongodb table 

In [1053]:
lesion_dataset_alta_ai = pd.read_csv('/data/oleksii/alta-ai.com/alta-ai-orthanc-backup/2023_08_15_full/segments.csv', sep=",")

def get_oid_from_uid(uid):
    study_candidates = orthanc_client.post_tools_lookup(data=uid)
    for i in study_candidates:
        if i['Type'] == 'Study':
            return i['ID']
    return None

lesion_dataset_alta_ai['study_orthanc_id'] = lesion_dataset_alta_ai.apply(lambda x: get_oid_from_uid(x['study_uid']), axis=1)

In [1057]:
# get available segmentations and match them with studyid 

# segmntations = glob.glob("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-new-2/*/*seg.nrrd")
segmentations = glob.glob("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-alta_ai-final-seg/*/Segmentation-label-altaai.seg.nrrd")
seg_oid = [seg_path.split('/')[-2] for seg_path in segmentations]

lesion_dataset_classes = lesion_dataset_alta_ai[lesion_dataset_alta_ai['study_orthanc_id'].isin(seg_oid)]

lesion_dataset_classes['segment_id'] = lesion_dataset_classes['segment_id'] + 1
lesion_dataset_classes['class'].replace('Chronic inflammation', 'CI', inplace=True)
lesion_dataset_classes['class'].replace('chronische Entzündung', 'CI', inplace=True)

lesion_dataset_classes['class'].replace('No tumor', 'N/A', inplace=True)
lesion_dataset_classes['class'].replace('Kein Tumor', 'N/A', inplace=True)
lesion_dataset_classes['class'].replace('High-grade PIN', 'HighPIN', inplace=True)

lesion_dataset_classes['class'].replace('Gleason 6', 'GS6', inplace=True)
lesion_dataset_classes['class'].replace('Gleason 7a', 'GS7a', inplace=True)
lesion_dataset_classes['class'].replace('Gleason 7b', 'GS7b', inplace=True)
lesion_dataset_classes['class'].replace('Gleason 8', 'GS8', inplace=True)
lesion_dataset_classes['class'].replace('Gleason 9', 'GS9', inplace=True)
lesion_dataset_classes['class'].replace('Gleason 10', 'GS10', inplace=True)
lesion_dataset_classes['class'].unique()

array(['CI', 'GS7a', 'GS7b', 'GS6', 'GS9', 'HighPIN', 'N/A', 'GS8',
       'GS10'], dtype=object)

In [1079]:
# group data
group = lesion_dataset_classes.groupby(['study_orthanc_id', 'series_uid'])
alta_ai_df = group.agg({'class': lambda x: ','.join([str(i)for i in x]), 
                    'region': lambda x: ','.join([str(i)for i in x]), 
                    'zone': lambda x: ','.join([str(i)for i in x]),
                    'segment_id': lambda x: ','.join([str(i)for i in x]),
                   })
alta_ai_df['count'] = group.count()['segment_id']
alta_ai_df = alta_ai_df.reset_index()
alta_ai_df = alta_ai_df.drop(columns=["series_uid"])


In [1080]:
alta_ai_df['class'].unique()

array(['CI', 'GS7b,GS7b', 'GS8', 'N/A,N/A', 'GS7a', 'GS7a,GS7a', 'N/A',
       'GS7b', 'GS6', 'GS7b,GS6', 'GS9,GS7a', 'GS6,GS6', 'GS9', 'CI,CI',
       'HighPIN', 'GS6,GS7a', 'GS9,GS7b', 'GS7b,HighPIN', 'GS7a,GS6',
       'GS7a,GS7a,GS7a', 'N/A,N/A,N/A,N/A', 'GS7b,GS7a', 'CI,CI,CI,CI,CI',
       'GS7b,CI', 'GS7b,GS7b,GS7a', 'GS8,GS7a', 'GS6,GS7a,GS7a',
       'GS7a,GS6,HighPIN,N/A', 'GS6,GS7a,GS6', 'GS10', 'GS7a,GS6,GS7a',
       'GS9,GS9', 'GS7a,GS6,CI', 'GS6,N/A', 'GS7a,GS7a,GS7b',
       'GS7a,GS7b,GS7a', 'CI,GS6', 'GS6,CI', 'GS6,GS9', 'GS7a,GS7b',
       'CI,N/A', 'GS6,GS6,GS6', 'GS7a,GS8', 'GS6,HighPIN', 'GS6,GS6,GS7a',
       'GS7a,HighPIN', 'GS7b,GS7a,GS7b', 'N/A,GS6', 'GS8,GS8', 'CI,GS7a',
       'GS7b,CI,GS7a', 'N/A,N/A,GS6', 'GS6,GS7b', 'GS9,GS6', 'GS9,GS8',
       'GS6,GS6,GS6,GS6', 'HighPIN,CI', 'GS8,GS9', 'GS7a,GS9',
       'GS7a,GS7a,GS6', 'GS7a,CI', 'GS8,GS6', 'GS7a,GS7b,GS7a,GS7a'],
      dtype=object)

In [1081]:
alta_ai_df

,study_orthanc_id,class,region,zone,segment_id,count
0,00459721-7e3e85cb-7a35fb08-66d5797f-50c34a9b,CI,Midglade,Peripher,1,1
1,0057f08b-a61b7ece-3d9edb9e-72196895-665428b3,"GS7b,GS7b","Basal,Midglade","Peripher,Transitional","1,2",2
2,0145941e-e51d6e7a-89dad7fc-cc086f24-387d3d86,GS8,Midglade,Peripher,1,1
3,020c80d6-b419794e-a5c282f9-cbe821e2-7082a06c,CI,Midglade,Peripher,1,1
4,02859d92-c129cf68-d5a65a3c-c7e6b1de-49552f73,"N/A,N/A","Basal,Basal","Peripher,Peripher","1,2",2
...,...,...,...,...,...,...
359,f9e7f547-04581881-8dcef6e2-908e88ff-1b080f5b,"GS9,GS7a","Midglade,Midglade","Peripher,Transitional","1,2",2
360,faeefe1a-452b6db8-4b9ef596-af856704-0fe1b1fa,"GS7b,GS7b","Midglade,Midglade","Whole Glade,Peripher","1,2",2
361,fb052d26-38d09433-eccfd044-06eaee8f-818d479f,GS7a,Midglade,Peripher,1,1
362,fcb50b30-a2045361-b1ebf099-1274fb7c-2f4df969,CI,Midglade,Peripher,1,1


In [1082]:
# populate with healthy patients

In [1083]:
segmentations_neg = glob.glob("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-negative-final-seg/*/Segmentation-label-*.seg.nrrd")
seg_oid = [seg_path.split('/')[-2] for seg_path in segmentations_neg]
lesion_dataset_negative = lesion_dataset_alta_ai[lesion_dataset_alta_ai['study_orthanc_id'].isin(seg_oid)]

In [1085]:
alta_ai_df_c = alta_ai_df.copy()
for oid in seg_oid:
    alta_ai_df_c = alta_ai_df_c.append({'study_orthanc_id': oid, 'class': 'N/A', 'count': 0.0}, ignore_index=True)

In [1224]:
alta_ai_df_c

,study_orthanc_id,class,region,zone,segment_id,count
0,00459721-7e3e85cb-7a35fb08-66d5797f-50c34a9b,CI,Midglade,Peripher,1,1.0
1,0057f08b-a61b7ece-3d9edb9e-72196895-665428b3,"GS7b,GS7b","Basal,Midglade","Peripher,Transitional","1,2",2.0
2,0145941e-e51d6e7a-89dad7fc-cc086f24-387d3d86,GS8,Midglade,Peripher,1,1.0
3,020c80d6-b419794e-a5c282f9-cbe821e2-7082a06c,CI,Midglade,Peripher,1,1.0
4,02859d92-c129cf68-d5a65a3c-c7e6b1de-49552f73,"N/A,N/A","Basal,Basal","Peripher,Peripher","1,2",2.0
...,...,...,...,...,...,...
860,76ebd33b-5396b8d5-ca5601d5-c5f5492c-4c5fa9a3,N/A,NaN,NaN,NaN,0.0
861,9022caf1-8a6ab505-88f49055-c2385f95-662fcd1c,N/A,NaN,NaN,NaN,0.0
862,f758b743-be8da533-b2100e2d-a2c97fbf-937e71aa,N/A,NaN,NaN,NaN,0.0
863,c52efb01-45f01d4b-3eee285c-1cf5d39b-d0431ea3,N/A,NaN,NaN,NaN,0.0


In [1269]:
final_lesion_dataset = pd.concat([alta_ai_df_c, real_class_available])
final_lesion_dataset = final_lesion_dataset.reset_index()
final_lesion_dataset = final_lesion_dataset.drop(columns=['index'])

In [1270]:
final_lesion_dataset['segment_id'] = final_lesion_dataset['segment_id'].apply(lambda x: ','.join([str(s) for s in [int(eval(i)) for i in x.split(',')]]) if not pd.isnull(x) else x)

In [1271]:
final_lesion_dataset

,study_orthanc_id,class,region,zone,segment_id,count
0,00459721-7e3e85cb-7a35fb08-66d5797f-50c34a9b,CI,Midglade,Peripher,1,1.0
1,0057f08b-a61b7ece-3d9edb9e-72196895-665428b3,"GS7b,GS7b","Basal,Midglade","Peripher,Transitional","1,2",2.0
2,0145941e-e51d6e7a-89dad7fc-cc086f24-387d3d86,GS8,Midglade,Peripher,1,1.0
3,020c80d6-b419794e-a5c282f9-cbe821e2-7082a06c,CI,Midglade,Peripher,1,1.0
4,02859d92-c129cf68-d5a65a3c-c7e6b1de-49552f73,"N/A,N/A","Basal,Basal","Peripher,Peripher","1,2",2.0
...,...,...,...,...,...,...
2015,ff62e719-e33423be-49a963b8-2d80a687-58e3d65c,"GS9,HighPIN,HighPIN",NaN,NaN,"1,2,3",3.0
2016,ff707595-1e66b158-1f36ab68-79289751-c171bb7f,"GS9,CI",NaN,NaN,"1,2",2.0
2017,ffcee9ca-5aa578e7-fdeac08a-949739d1-728027eb,CI,NaN,NaN,1,1.0
2018,ffcf7f45-fd4a6512-b553774b-5f0de689-c17cfe46,GS7b,NaN,NaN,1,1.0


In [1290]:
def is_csPCa(x):
    csPCa_list_ = ['GS7a', 'GS7b', 'GS8', 'GS9', 'GS10']
    return any(word in x for word in csPCa_list_)

final_lesion_dataset['case_csPCa'] = final_lesion_dataset['class'].apply(lambda x: is_csPCa(x) if not pd.isnull(x) else x)

In [1307]:
def max_class(x):
    ordered_classes = ['N/A', 'HighPIN', 'CI', 'GS6', 'GS7a', 'GS7b', 'GS8', 'GS9', 'GS10']
    max_class = max([ordered_classes.index(cl) for cl in x.split(',')])
    return ordered_classes[max_class]

final_lesion_dataset['case_class'] = final_lesion_dataset['class'].apply(lambda x: max_class(x) if not pd.isnull(x) else x)

In [1321]:
segmentations_neg = glob.glob("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-negative-final-seg/*/Segmentation-label-*.seg.nrrd")
seg_oid = [seg_path.split('/')[-2] for seg_path in segmentations_neg]
final_lesion_dataset['healthy'] = np.where(final_lesion_dataset['study_orthanc_id'].isin(seg_oid), True, False)

In [1328]:
# populate with extra information
for i, row in final_lesion_dataset.iterrows():
    study_main_info = Study(row['study_orthanc_id'], orthanc_client).get_main_information()
    study_date = study_main_info['MainDicomTags']['StudyDate']
    study_UID = study_main_info['MainDicomTags']['StudyInstanceUID']
    patient_id = study_main_info['PatientMainDicomTags']['PatientID']
    final_lesion_dataset.loc[i, 'study_date'] = study_date
    final_lesion_dataset.loc[i, 'patient_id'] = patient_id
    final_lesion_dataset.loc[i, 'StudyInstanceUID'] = study_UID

In [1329]:
final_lesion_dataset.to_csv(f'meta_data_lesion/lesion_dataset_2020_{today}.csv', sep=';', index=False)

## Merge folders

In [1391]:
os.makedirs(destination, exist_ok=True)

In [1418]:
import shutil
target = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-negative-final-seg"
destination = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-final-seg"

# target = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-negative-final"
# destination = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-final"

In [1420]:
for soid in os.listdir(target):
    target_path = os.path.join(target, soid)
    destination_path = os.path.join(destination, soid)
#     print(target_path, destination_path)
#     shutil.move(target_path, destination_path)

FileNotFoundError: [Errno 2] No such file or directory: '/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-negative-final-seg'

In [1549]:
studies = os.listdir("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset")
check_true = final_lesion_dataset[final_lesion_dataset['study_orthanc_id'].isin(studies)]

In [1460]:
df_duplicated = check_true[check_true.duplicated(subset=['study_orthanc_id'], keep=False)].sort_values('StudyInstanceUID')

In [1500]:
len(df_duplicates_processed)

157

In [1496]:
df_duplicates_processed = pd.read_csv('meta_data_lesion/duplicates_OB.csv', sep=';')
keep_manual_rows = df_duplicates_processed[df_duplicates_processed['Keep']=='TRUE']
df_duplicates_processed_2 = df_duplicates_processed[~df_duplicates_processed['study_orthanc_id'].isin(keep_fraction['study_orthanc_id'])]
max_indices = df_duplicates_processed_2.groupby('study_orthanc_id')['count'].idxmax()
selected_rows = df_duplicates_processed_2.loc[max_indices]

In [1556]:
final_result = final_result.drop(columns=['Unnamed: 12', 'Keep', 'Unnamed: 14'])

In [1516]:
final_result = pd.concat([selected_rows, keep_manual_rows]).sort_index()
final_result.to_csv(f'meta_data_lesion/solved_duplicates.csv', sep=';', index=False)

In [1544]:
oids = [path.split('/')[-2] for path in glob.glob("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset-seg/*/*/*.seg.nrrd")]


In [1458]:
# # delete duplicated images 
# img_dir = "/data/oleksii/Prostate-Lesion-Datasets-NRRDS/ALTA-Lesion-Dataset"

# for study in df_duplicated[df_duplicated.duplicated(subset=['study_orthanc_id'])]['study_orthanc_id'].tolist():
#     dupl_path = os.path.join(img_dir, study, study)
#     if os.path.exists(dupl_path):
#         shutil.rmtree(dupl_path)


In [1570]:
final_lesion_dataset_1 = final_lesion_dataset[~final_lesion_dataset['study_orthanc_id'].isin(final_result['study_orthanc_id'])]


final_final_lesion_dataset = pd.concat([final_lesion_dataset_1, final_result]).sort_values(by='study_orthanc_id')

In [1571]:
final_final_lesion_dataset = final_final_lesion_dataset.reset_index(drop=True)

In [1573]:
final_final_lesion_dataset
final_final_lesion_dataset.to_csv(f'meta_data_lesion/lesion_dataset_{len(final_final_lesion_dataset)}_{today}.csv', sep=';', index=False)

## Extract zone info

In [1379]:
# Zonen info

segmentations_zonal = glob.glob("/data/oleksii/Prostate-ZONE-Datasets-NRRDS/ALTA-Zone-Dataset/*/")
seg_oid = [seg_path.split('/')[-2] for seg_path in segmentations_zonal]

In [1361]:
from pyorthanc import Series
import urllib.request
import urllib.error
import httpx


In [1374]:
some_missing = []
records = []

for soid in seg_oid:
    d = {}

    try:
        series = Series(soid, orthanc_client)
        study = Study(series.study_identifier, orthanc_client)
        main_info = study.get_main_information()
        study_date = main_info['MainDicomTags']['StudyDate']
        study_UID = main_info['MainDicomTags']['StudyInstanceUID']
        patient_id = main_info['PatientMainDicomTags']['PatientID']
        
        d['study_date'] = study_date
        d['patient_id'] = patient_id
        d['StudyInstanceUID'] = study_UID
        d['study_orthanc_id'] = study.id_
        d['t2w_series_oid'] = soid

        records.append(d)

    except httpx.HTTPError as e:
#         print(f'A HTTPError was thrown: {e} ')
        some_missing.append(soid)
    
metadata_zone = pd.DataFrame(records)

In [1378]:
metadata_zone.to_csv('zone_dataset_1260_20240129.cvs', sep=';', index=False)

In [ ]:
len(some_missing)